In [0]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [0]:
# Create tensor
x=torch.tensor(5., requires_grad=True)
w=torch.tensor(2., requires_grad=True)
b=torch.tensor(3., requires_grad=True)

In [0]:
# Build a computational graph
y=w*x + b

In [0]:
# Compute gradient
y.backward()

In [0]:
# Print out the gradients
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(5.)
tensor(1.)


In [0]:
#Create tensors of shape (10,3) and (10,2)
x=torch.randn(10,3)
y=torch.randn(10,2)

In [0]:
# Build a fully connected Layer
linear=nn.Linear(3,2)
print('w:', linear.weight)
print('b:', linear.bias)

w: Parameter containing:
tensor([[-0.3200, -0.1834, -0.2139],
        [ 0.4374,  0.1262, -0.5244]], requires_grad=True)
b: Parameter containing:
tensor([-0.4017,  0.4675], requires_grad=True)


In [0]:
# Build loss function and Optimizer
criterion=nn.MSELoss()
optimizer=torch.optim.SGD(linear.parameters(), lr=0.01)

In [0]:
# Forward pass
pred=linear(x)

In [0]:
# Compute loss
loss=criterion(pred,y)
print('loss:', loss.item())

loss: 2.20231556892395


In [0]:
# Backward Pass
loss.backward()

In [0]:
# Print out the gradients
print('dL/dw:', linear.weight.grad)
print('dL/db:', linear.bias.grad)

dL/dw: tensor([[-0.0286,  0.2967, -0.3998],
        [ 1.8837,  0.0100, -1.2629]])
dL/db: tensor([-0.4250, -0.0871])


In [0]:
# 1-step gradient descent
optimizer.step()

In [0]:
pred=linear(x)

In [0]:
loss=criterion(pred,y)

In [0]:
print('loss after 1 step optimization=', loss.item())

loss after 1 step optimization= 2.1471140384674072


Loading data from Numpy

In [0]:
# create numpy array
x=np.array([[1,2],[3,4]])

In [0]:
# convert numpy array to torch tensor
y=torch.from_numpy(x)

In [0]:
# convert torch tensor to numpy array
z=y.numpy()

In [0]:
z.shape

(2, 2)

                                  ########### 4. Input Pipline

In [0]:
# Download and construct the CIFAR 10 dataset
train_dataset=torchvision.datasets.CIFAR10(root= '../../data',
                                                               train= True,
                                                               transform=transforms.ToTensor() ,
                                                               download= True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

170500096it [00:02, 71595616.40it/s]                               


In [0]:
# Fetch one data pair (read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


In [0]:
# Data loader (it provides queues and threads in a very simple ways)
train_loader=torch.utils.data.DataLoader(dataset= train_dataset,
                                        batch_size= 64,
                                        shuffle= True)

In [0]:
# When iteration starts, queue and thread start to load data from files.
data_iter=iter(train_loader)

In [0]:
#mini-batch images and labels
images, labels= data_iter.next()

In [0]:
#Actual usage of dataloader is as bellow
for i, (images, labels) in enumerate(train_loader):
  pass

781

 #####################   5- Input pipline for custom datasets

In [0]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
         pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 1

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

################### 6- Load Pre-trained Model

In [0]:
# Download and load the pretrained ResNet-18.

resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:00<00:00, 111665223.22it/s]


torch.Size([64, 100])


####################       7- Save and Load mode

In [0]:
#save and load the entire model
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

In [0]:
# save and load the models parametrs (recomended)
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])